In [20]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import random
import os

In [21]:
def fix_seed(seed):
    # random
    random.seed(seed)
    # Numpy
    np.random.seed(seed)

SEED = 42
fix_seed(SEED)

ESサイトのユーザーに対して**RCT**を適用したメールマーケティングを行ったデータ。

In [22]:
# データのロード
base_path = "/Users/ryoto/workspace/hit-u/zemi/econml_demo/data"
mail = pd.read_csv(os.path.join(base_path,'E-MailAnalytics.csv'))
print(mail.shape)
mail.head()

(64000, 12)


,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend
0,10,2) $100 - $200,142.44,1,0,Surburban,0,Phone,Womens E-Mail,0,0,0.0
1,6,3) $200 - $350,329.08,1,1,Rural,1,Web,No E-Mail,0,0,0.0
2,7,2) $100 - $200,180.65,0,1,Surburban,1,Web,Womens E-Mail,0,0,0.0
3,9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0.0
4,2,1) $0 - $100,45.34,1,0,Urban,0,Web,Womens E-Mail,0,0,0.0


In [23]:
print("num of No E-Mail : ", len(mail.query("segment == 'No E-Mail'")))
print("num of Mens E-Mail : ", len(mail.query("segment == 'Womens E-Mail'")))
print("num of No Womens : ", len(mail.query("segment == 'Mens E-Mail'")))

num of No E-Mail :  21306
num of Mens E-Mail :  21387
num of No Womens :  21307


## RCTのデータからATEを求めてBaseLineとする
簡略化のために女性向けのメールが配信されているデータを削除する。

In [24]:
mail_df = mail.query("segment != 'Womens E-Mail'")
print(len(mail_df.query("segment == 'No E-Mail'")))
print(len(mail_df.query("segment == 'Womens E-Mail'")))
print(len(mail_df.query("segment == 'Mens E-Mail'")))

21306
0
21307


In [25]:
mail_df["segment"] = mail_df.segment.map(lambda x: 1 if x == 'Mens E-Mail' else 0)
mail_df = mail_df.rename(columns={"segment" : "treatment"})
mail_df.head()


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,recency,history_segment,history,mens,womens,zip_code,newbie,channel,treatment,visit,conversion,spend
1,6,3) $200 - $350,329.08,1,1,Rural,1,Web,0,0,0,0.0
3,9,5) $500 - $750,675.83,1,0,Rural,1,Web,1,0,0,0.0
8,9,5) $500 - $750,675.07,1,1,Rural,1,Phone,1,0,0,0.0
13,2,2) $100 - $200,101.64,0,1,Urban,0,Web,1,1,0,0.0
14,4,3) $200 - $350,241.42,0,1,Rural,1,Multichannel,0,0,0,0.0


In [26]:
mail_df.groupby("treatment").agg({"spend" : "mean", "conversion": "mean", "visit":"count"}).rename(columns={"visit" : "count"})

,spend,conversion,count
treatment,,,
0,0.652789,0.005726,21306
1,1.422617,0.012531,21307


In [27]:
# treatment 0, 1 のデータフレームに分ける
treatment_1 = mail_df.query("treatment == 1")
treatment_0 = mail_df.query("treatment == 0")

In [28]:
# 介入が購買金額に与えた影響を計算する
ts_1 = treatment_1["spend"].mean()
ts_0 = treatment_0["spend"].mean()
print("介入が購買金額に与えた影響は,", np.round(ts_1 - ts_0, 3))

介入が購買金額に与えた影響は, 0.77


In [29]:
# 介入がconversionに与えた影響を計算する
tc_1 = treatment_1["conversion"].mean()
tc_0 = treatment_0["conversion"].mean()
True_ATE = np.round((tc_1 - tc_0), 5)
print("介入がconversionに与えた影響は,", True_ATE)

介入がconversionに与えた影響は, 0.00681


## データの前処理

In [30]:
mail_df.head()

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,treatment,visit,conversion,spend
1,6,3) $200 - $350,329.08,1,1,Rural,1,Web,0,0,0,0.0
3,9,5) $500 - $750,675.83,1,0,Rural,1,Web,1,0,0,0.0
8,9,5) $500 - $750,675.07,1,1,Rural,1,Phone,1,0,0,0.0
13,2,2) $100 - $200,101.64,0,1,Urban,0,Web,1,1,0,0.0
14,4,3) $200 - $350,241.42,0,1,Rural,1,Multichannel,0,0,0,0.0


In [31]:
# ラベルエンコーディング（OrdinalEncoder）
from sklearn.preprocessing import OrdinalEncoder

oe = OrdinalEncoder()
encoded = oe.fit_transform(mail_df[['history_segment', 'zip_code', "channel"]].values)
# decoded = oe.inverse_transform(encoded)

print('エンコード結果: ')
encoded_df = pd.DataFrame(encoded, columns = ["history_segment", "zip_code", "channel"])
encoded_df.head()

エンコード結果: 


,history_segment,zip_code,channel
0,2.0,0.0,2.0
1,4.0,0.0,2.0
2,4.0,0.0,1.0
3,1.0,2.0,2.0
4,2.0,0.0,0.0


In [32]:
mail_df_use = mail_df[
    ['recency', 'history', 'mens', 'womens', 'newbie', 'treatment', 'visit', 'conversion', 'spend']
    ]
demo_df = pd.concat([mail_df_use.reset_index(), encoded_df], axis = 1).drop(columns = "index").sample(2500)
print(demo_df.shape)
demo_df.head()

(2500, 12)


,recency,history,mens,womens,newbie,treatment,visit,conversion,spend,history_segment,zip_code,channel
322,4,121.87,1,0,0,0,0,0,0.0,1.0,0.0,2.0
14339,3,184.83,1,0,1,1,0,0,0.0,1.0,2.0,1.0
2348,2,400.48,0,1,1,0,0,0,0.0,3.0,2.0,0.0
33454,10,175.25,0,1,0,1,1,0,0.0,1.0,0.0,2.0
25505,2,1640.42,1,1,1,1,0,0,0.0,6.0,0.0,0.0


In [33]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(demo_df, test_size= 0.95,random_state=SEED)
print(train_df.shape)
print(val_df.shape)

(125, 12)
(2375, 12)


## Econmlで試す

In [34]:
# Main imports
from econml.metalearners import TLearner, SLearner, XLearner, DomainAdaptationLearner

# Helper imports 
import numpy as np
from numpy.random import binomial, multivariate_normal, normal, uniform
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, GradientBoostingClassifier
import matplotlib.pyplot as plt

In [35]:
# set data 
Y = train_df.conversion
T = train_df.treatment
X = train_df.drop(columns=["conversion", "treatment"])
X_val = val_df.drop(columns=["conversion", "treatment"])

### T-Learner

In [41]:
# Instantiate T learner
models = RandomForestClassifier(n_estimators=5000,  random_state = 42)
T_learner = TLearner(models=models)
# Train T_learner
T_learner.fit(Y, T, X=X)
# Estimate treatment effects on test data
T_te = T_learner.effect(X_val)

print("True ATE : ", True_ATE)
pred_ATE_T = np.round(T_learner.ate(X_val), 5)
print("Predict ATE : ", pred_ATE_T)

True ATE :  0.00681
Predict ATE :  0.0


### S-Learner

In [37]:
# Instantiate S learner
overall_model = RandomForestClassifier(n_estimators=5000, random_state = 42)
S_learner = SLearner(overall_model=overall_model)
# Train S_learner
S_learner.fit(Y, T, X=X)
# Estimate treatment effects on test data
S_te = S_learner.effect(X_val)

print("True ATE : ", True_ATE)
pred_ATE_S = np.round(S_learner.ate(X_val), 5)
print("Predict ATE : ", pred_ATE_S)

True ATE :  0.00681
Predict ATE :  0.0


### X-Learner

In [38]:
# Instantiate X learner
models = RandomForestClassifier(n_estimators=5000, random_state = 42)
propensity_model = RandomForestClassifier(n_estimators=5000, random_state = 42)
X_learner = XLearner(models=models, propensity_model=propensity_model)
# Train X_learner
X_learner.fit(Y, T, X=X)
# Estimate treatment effects on test data
X_te = X_learner.effect(X_val)
print("True ATE : ", True_ATE)
pred_ATE_X = np.round(X_learner.ate(X_val), 5)
print("Predict ATE : ", pred_ATE_X)

True ATE :  0.00681
Predict ATE :  0.0
